## 各種import

In [1]:
from io import StringIO
from pyxcp.config import readConfiguration
from pyxcp import transport
from pyxcp.transport.candriver.pc_vector import Vector # "CAN_DRIVER": "Vector"を指定する場合に必要
from pyxcp.master import Master
from collections import namedtuple
import time

## コンフィグレーションパラメータ(JSON文字列)の作成と読み込み

In [2]:
JSON = """{
"TRANSPORT": "CAN",
"CAN_DRIVER": "Vector",
"CAN_USE_DEFAULT_LISTENER": true,
"CHANNEL": "0",
"CAN_ID_MASTER": 2,
"CAN_ID_SLAVE": 1,
"CAN_ID_BROADCAST": 256,
"MAX_DLC_REQUIRED": false,
"CREATE_DAQ_TIMESTAMPS": true,
"SERIAL": 0,
"BITRATE": 500000,
"APP_NAME": "PyXCP-CAN"
}"""
CONF_JSON = StringIO(JSON)
CONF_JSON.name = "conf_can_vector.json"
CONFIG = readConfiguration(CONF_JSON)

## コンフィグレーションを元にXCPマスターの生成

In [3]:
xm=Master('can',config=CONFIG)

## CONNECT

In [4]:
conn = xm.connect()
print(conn)

Container: 
    resource = Container: 
        dbg = False
        pgm = False
        stim = True
        daq = True
        calpag = False
    commModeBasic = Container: 
        optional = True
        slaveBlockMode = False
        addressGranularity = (enum) BYTE 0
        byteOrder = (enum) INTEL 0
    maxCto = 8
    maxDto = 8
    protocolLayerVersion = 16
    transportLayerVersion = 16


In [5]:
print("calpag =", xm.slaveProperties.supportsCalpag)
print("daq =", xm.slaveProperties.supportsDaq)
print("pgm =", xm.slaveProperties.supportsPgm)
print("stim =", xm.slaveProperties.supportsStim)

calpag = False
daq = True
pgm = False
stim = True


## GET_STATUS

In [6]:
status=xm.getStatus()
print(status)

Container: 
    sessionStatus = Container: 
        resume = False
        daqRunning = False
        clearDaqRequest = False
        storeDaqRequest = False
        storeCalRequest = False
    resourceProtectionStatus = Container: 
        dbg = False
        pgm = False
        stim = False
        daq = False
        calpag = False
    sessionConfiguration = 0


## SYNCH

In [7]:
res=xm.synch()
print(res)

bytearray(b'\x00')


## GET_COMM_MODE_INFO

In [8]:
CommModeInfo=xm.getCommModeInfo()
print(CommModeInfo)

Container: 
    commModeOptional = Container: 
        interleavedMode = False
        masterBlockMode = False
    maxBs = 99
    minSt = 0
    queueSize = 99
    xcpDriverVersionNumber = 16


## DOWNLOAD、UPLOADによるverify

In [9]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(4)
print(bhv.hex())

00000000


In [10]:
bhv = bytearray(bhv)
bhv[0] = 0xAA
bhv[1] = 0x55
xm.setMta(0x0000,0xff)
xm.download(bhv)

bytearray(b'')

In [11]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(4)
print(bhv.hex())

aa550000


## DAQリストの構築

In [12]:
xm.freeDaq();

In [13]:
xm.allocDaq(2);

In [14]:
xm.allocOdt(0, 1);
xm.allocOdt(1, 1);

In [15]:
xm.allocOdtEntry(0, 0, 1);
xm.allocOdtEntry(1, 0, 1);

## ODT_ENTRYへの書き込み

In [16]:
DaqEntry = namedtuple("DaqEntry", "daq odt entry bitoff size ext addr")
de0 = (
    DaqEntry(daq=0, odt=0,  entry=0, bitoff=255, size=4, ext=0xff, addr=0x00000004),
    DaqEntry(daq=1, odt=0,  entry=0, bitoff=255, size=4, ext=0xff, addr=0x00000004),
)


In [17]:
for daq, odt, entry, bitoff, size, ext, addr in de0:
    xm.setDaqPtr(daq, odt,  entry)
    xm.writeDaq(bitoff, size, ext, addr)

## DAQの起動

In [18]:
xm.setDaqListMode(0x10, 0, 1, 1, 0)
xm.setDaqListMode(0x02, 1, 3, 1, 0)

res = xm.startStopDaqList(0x02, 0)
print("startStopDaqList #0", res)
res = xm.startStopDaqList(0x02, 1)
print("startStopDaqList #1", res)

startStopDaqList #0 Container: 
    firstPid = 0
startStopDaqList #1 Container: 
    firstPid = 1


In [19]:
xm.startStopSynch(0x01);

## DAQの計測とSTIMによる外部からの値更新

In [20]:
stim_wait = 0.05
start = time.time()
stim_time =  time.time()+stim_wait-0.03

ram_update = [
    [0x11, 0x22, 0x33, 0x44],
    [0x55, 0x66, 0x77, 0x88],
    [0x99, 0xAA, 0xBB, 0xCC],
    [0x00, 0x00, 0x00, 0x00],
]
i = 0
str = ""
while True:
    queue_len = len(xm.transport.daqQueue)
    for _ in range(queue_len):
        daq = xm.transport.daqQueue.popleft()
        str += '%.6f, %s\n' % ( daq[3],daq[0].hex() )
    if time.time() > stim_time:
        bhv = bytearray([0x01]*5)
        bhv[1:5]=ram_update[i]
        xm.transport.send(bhv)
        str += 'stim %s\n' % bhv.hex()
        stim_time =  stim_time + stim_wait
        i = i+1
        if i > 3:
            i = 3
    if time.time() > start + 0.22:
        break
    time.sleep(0.010)

print(str)

1641028043.164187, 00c0b000000000
1641028043.174189, 00cab000000000
1641028043.184159, 00d4b000000000
1641028043.194112, 00deb000000000
1641028043.204107, 00e8b000000000
1641028043.214068, 00f2b000000000
stim 0111223344
1641028043.224030, 00fcb000000000
1641028043.234032, 0006b111223344
1641028043.244018, 0010b111223344
1641028043.253979, 001ab111223344
1641028043.263925, 0024b111223344
stim 0155667788
1641028043.273911, 002eb155667788
1641028043.283946, 0037b155667788
1641028043.294841, 0042b155667788
1641028043.304835, 004cb155667788
stim 0199aabbcc
1641028043.314879, 0056b155667788
1641028043.324783, 0060b199aabbcc
1641028043.334818, 006ab199aabbcc
1641028043.344747, 0074b199aabbcc
1641028043.354758, 007eb199aabbcc
stim 0100000000
1641028043.364678, 0088b199aabbcc
1641028043.374705, 0092b100000000
1641028043.384617, 009cb100000000
1641028043.394620, 00a6b100000000
1641028043.404557, 00b0b100000000
stim 0100000000



In [21]:
xm.startStopSynch(0x00);

In [22]:
xm.disconnect();